Exercise 1

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# First point

In [63]:
# Leer los datos del archivo CSV
clientes_data = pd.read_csv('/Users/brayanbaez/bi/datos_de_clientes.csv', sep='\t')

#Arregla los datos de estado marital, quitando los valores atipicos 

# Reemplazar los valores "YOLO" y "Absurd" por "Married"
clientes_data['Marital_Status'] = clientes_data['Marital_Status'].replace(['YOLO', 'Absurd'], 'Married')

# Verificar los cambios
print(clientes_data['Marital_Status'].unique())


# Diccionarios de mapeo para codificación ordinal
education_mapping = {'Basic': 1, '2n Cycle': 2, 'Graduation': 3, 'Master': 4, 'PhD': 5}
marital_status_mapping = {'Alone': 1, 'Widow': 2, 'Divorced': 3, 'Single': 4, 'Together': 5, 'Married': 6}

# Aplicar la codificación ordinal y reemplazar las columnas originales
clientes_data['Education'] = clientes_data['Education'].map(education_mapping)
clientes_data['Marital_Status'] = clientes_data['Marital_Status'].map(marital_status_mapping)

#Revisar si hay espacio vacios
# Iterar sobre las filas del DataFrame
for index, row in clientes_data.iterrows():
    # Verificar si hay un valor nulo en la columna 'ID'
    if pd.isnull(row['ID']):
        print("Hay un espacio vacío en el ID", index)
  

# Convertir la columna 'Dt_Customer' a tipo datetime
clientes_data['Dt_Customer'] = pd.to_datetime(clientes_data['Dt_Customer'], format='%d-%m-%Y')

# Definir la función lambda para codificar las fechas
def codificar_fecha(fecha):
    if fecha.year < 2013:
        return 1
    elif fecha.year == 2013:
        return 2
    else:
        return 3

# Aplicar la función lambda a la columna 'Dt_Customer'
clientes_data['Dt_Customer'] = clientes_data['Dt_Customer'].apply(lambda x: codificar_fecha(x))

# Mostrar los primeros registros con la columna 'Dt_Customer' codificada
print(clientes_data.head())





['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone']
     ID  Year_Birth  Education  Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957          3               4  58138.0        0         0   
1  2174        1954          3               4  46344.0        1         1   
2  4141        1965          3               5  71613.0        0         0   
3  6182        1984          3               5  26646.0        1         0   
4  5324        1981          5               6  58293.0        1         0   

   Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0            1       58       635  ...                  7             0   
1            3       38        11  ...                  5             0   
2            2       26       426  ...                  4             0   
3            3       26        11  ...                  6             0   
4            3       94       173  ...                  5             0   

   AcceptedCmp4  Acce